# Analysis of Fading-Memory Systems with Higher Order Statistics

We will have a look at practical issues when analysing fading memory systems
higher order statistics.

So let's get started. First, we need some imports.

In [1]:
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as ssig
from scipy.linalg import hankel
from stingray.bispectrum import Bispectrum
from stingray.lightcurve import Lightcurve

from src.utils.plotting import init_plot_style
# %pylab

# initialize our global plot style
init_plot_style()

/home/sophie/miniconda3/envs/nsp2021/lib/python3.7/site-packages/stingray/crossspectrum.py:21: UserWarning: Using standard scipy fft
  warnings.warn("Using standard scipy fft")


We define a linear and a nonlinear fading memory system for later analysis.

In [2]:
def linsys(x, order = 5, cutoff = 0.3):
    """Implements a linear, causal IIR filter..

    Parameters
    ----------
    x : float or numpy.ndarray
        The systems input.

    Returns
    ------
    float or numpy.ndarray
        The corresponding system output.
    """
    b, a = ssig.butter(order, cutoff)
    y = ssig.lfilter(b, a, x)
    return y

def system(x):
    """Implements a deterministic, fading memory nonlinearity.

    To generate an output of equal length as the input signal we apply zero-padding.

    Parameters
    ----------
    x : float or numpy.ndarray
        The systems input.

    Returns
    ------
    float or numpy.ndarray
        The corresponding system output.
    """
    mem_depth = 5
    weights = np.array([1.1, 4.5, 1.4, 1.3, 3.2])
    padded_x = np.concatenate((np.zeros(mem_depth - 1,), x))
    X = hankel(padded_x[:mem_depth], padded_x[mem_depth-1:])
    X[0,:] = (X[0,:] - X[4,:])**2
    X[1,:] = np.exp(-np.abs(X[1,:] - X[2,:]))
    X[2,:] = np.sqrt(np.abs(X[2,:]))
    X[3,:] = X[3,:] *  X[4,:]
    X[4,:] = np.log(np.abs(X[4,:]) + 1e-3)
    out = weights.dot(X)
    return out - np.mean(out)


Here we make a simple correlation analysis.

In [3]:
# generate input and output signals
n_samples = int(1e3)
x = np.random.randn(n_samples)
y = linsys(x, order=3)
# y = system(x)

# compute correlation function
max_lag = 20
xcorr = ssig.correlate(y, y, mode='same') / n_samples
lags = ssig.correlation_lags(n_samples, n_samples, mode='same')

# plot the correlation function
plt.close('all')
plt.figure()
plt.stem(lags[np.abs(lags)<max_lag], xcorr[np.abs(lags)<max_lag])
plt.xlabel('Time Lag, $k$')
plt.ylabel('Autocorrelation')
plt.tight_layout()


AttributeError: module 'scipy.signal' has no attribute 'correlation_lags'

Next, we have a look at second-order statistics: (cross) power spectral density and coherence.


In [ ]:
# generate input and output signals
n_samples = int(1e3)
x = np.random.randn(n_samples)
y = linsys(x, order=3)
# y = system(x)

# estimation parameters
win = 'rect'
win_length = 512
nfft = max(win_length, 1024)

# estimate psd, cpsd and coherence
f_psd, psd = ssig.welch(y, nperseg=win_length, nfft=nfft, window=win)
f_csd, csd = ssig.csd(x, y, nperseg=win_length, nfft=nfft, window=win)
f_coh, coh = ssig.coherence(x, y, nperseg=win_length, nfft=nfft, window=win)

# plot the spectra
plt.close('all')
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(f_psd[1:], np.abs(psd)[1:], label='PSD(y)')
ax1.plot(f_csd[1:], np.abs(csd)[1:], label='CPSD(x, y)')
ax1.semilogy()
# ax1.set_ylim([1e-8, 1e1])
ax2.plot(f_coh, np.abs(coh), 'seagreen')
ax2.set_ylabel('Coherence', color='seagreen')
ax2.tick_params(axis='y', labelcolor='seagreen')
plt.xlabel(r'Normalized Frequency, $\theta$')
ax1.legend(loc=1)
plt.tight_layout()


Finally, we have a look at bispectra!


In [ ]:
# generate input and output signals
n_samples = int(1e4)
x = np.random.randn(n_samples)
y = linsys(x, order=3)
# y = system(x)

# estimation parameters
win = 'welch'
max_lag = 256
nfft = max(win_length, 1024)

lc = Lightcurve(np.arange(n_samples), y, dt=1, skip_checks=True)
bs = Bispectrum(lc, maxlag=max_lag, window=win, scale='unbiased')

# plot the bispectrum
plt.close('all')
fig, ax = plt.subplots()
bs.plot_mag()